In [15]:

import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import os
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## stop words

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [23]:
data = document = open(os.path.join('abstracts_collection.txt')).readlines()

In [27]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['we', 'investigate', 'various', 'contextual', 'effects', 'on', 'text', 'interpretation', 'and', 'account', 'for', 'them', 'by', 'providing', 'contextual', 'constraints', 'in', 'logical', 'theory', 'of', 'text', 'interpretation', 'on', 'the', 'basis', 'of', 'the', 'way', 'these', 'constraints', 'interact', 'with', 'the', 'other', 'knowledge', 'sources', 'we', 'draw', 'some', 'general', 'conclusions', 'about', 'the', 'role', 'of', 'domain', 'specific', 'information', 'top', 'down', 'and', 'bottom', 'up', 'discourse', 'information', 'flow', 'and', 'the', 'usefulness', 'of', 'formalisation', 'in', 'discourse', 'theory', 'introduct', 'ion', 'and', 'two', 'essential', 'parts', 'of', 'discourse', 'interpretation', 'involve', 'determining', 'the', 'rhetorical', 'role', 'each', 'sentence', 'plays', 'in', 'the', 'text', 'and', 'ii', 'determining', 'the', 'temporal', 'relations', 'between', 'the', 'events', 'described', 'preceding', 'discourse', 'context', 'has', 'significant', 'effects', 'on',

In [29]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['we', 'investigate', 'various', 'contextual', 'effects', 'on', 'text', 'interpretation', 'and', 'account', 'for', 'them', 'by', 'providing', 'contextual', 'constraints', 'in', 'logical', 'theory', 'of', 'text', 'interpretation', 'on', 'the', 'basis', 'of', 'the', 'way', 'these', 'constraints', 'interact', 'with', 'the', 'other', 'knowledge', 'sources', 'we', 'draw', 'some', 'general', 'conclusions', 'about', 'the', 'role', 'of', 'domain', 'specific', 'information', 'top_down', 'and', 'bottom_up', 'discourse', 'information', 'flow', 'and', 'the', 'usefulness', 'of', 'formalisation', 'in', 'discourse', 'theory', 'introduct', 'ion', 'and', 'two', 'essential', 'parts', 'of', 'discourse', 'interpretation', 'involve', 'determining', 'the', 'rhetorical', 'role', 'each', 'sentence', 'plays', 'in', 'the', 'text', 'and', 'ii', 'determining', 'the', 'temporal', 'relations', 'between', 'the', 'events', 'described', 'preceding', 'discourse', 'context', 'has', 'significant', 'effects', 'on', 'both'

In [31]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [33]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['investigate', 'various', 'contextual', 'effect', 'text', 'interpretation', 'account', 'provide', 'contextual', 'constraint', 'logical', 'theory', 'text', 'interpretation', 'basis', 'way', 'constraint', 'interact', 'knowledge', 'source', 'draw', 'general', 'conclusion', 'role', 'domain', 'specific', 'information', 'top', 'bottom', 'discourse', 'information', 'flow', 'usefulness', 'formalisation', 'discourse', 'theory', 'introduct', 'ion', 'essential', 'part', 'discourse', 'interpretation', 'involve', 'determine', 'rhetorical', 'role', 'sentence', 'play', 'text', 'determine', 'temporal', 'relation', 'event', 'describe', 'precede', 'discourse', 'context', 'significant', 'effect', 'aspect', 'interpretation', 'example', 'text', 'vacuo', 'may', 'non', 'iconic', 'explanation', 'pushing', 'cause', 'falling', 'explain', 'max', 'fall', 'pair', 'sentence', 'may', 'receive', 'iconic', 'narrative', 'interpretat', 'ion', 'discourse', 'context', 'provide', 'takes_advantage', 'max', 'vulnerability'

## Adding stop words

In [72]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
print(len(id2word))
# remove useless tokens
list_of_ids_to_remove = [id2word.token2id[tok] for tok in 'research,use,show,paper,model,base,result,method,approach,propose,problem,algorithm,present,datum,also,new,experiment,provide,work,good,demonstrate,improve,give,first,different,novel,achieve,however,many,well,introduce,art'.split(',')]
id2word.filter_tokens(list_of_ids_to_remove)
print(len(id2word))
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

24023
23991
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 2), (16, 3), (17, 2), (18, 1), (19, 2), (20, 5), (21, 1), (22, 1), (23, 1), (24, 3), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 3), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 1), (47, 4), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 7), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 3), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 5), (86, 1), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1)]]


In [74]:
i_f_tuples = list(id2word.dfs.items())

In [75]:
i_f_tuples_sorted = sorted(i_f_tuples, key=lambda x:x[1], reverse = True)


In [76]:
for i, f in i_f_tuples_sorted[:100]:
    print(id2word[i])

learn
task
system
set
performance
feature
information
language
state
word
large
machine
learning
structure
time
number
describe
analysis
technique
study
framework
domain
apply
make
training
text
function
exist
process
application
order
perform
compare
several
representation
find
semantic
outperform
high
dataset
develop
consider
evaluation
include
accuracy
space
simple
experimental
previous
classification
require
evaluate
combine
translation
sentence
efficient
knowledge
train
obtain
generate
allow
class
corpus
solve
natural
may
example
general
level
english
linear
non
important
test
network
address
statistical
multiple
standard
significantly
context
case
label
often
search
solution
distribution
human
represent
way
automatically
term
identify
focus
know
measure
inference
challenge
call
type


## Add Mallet path

In [70]:
mallet_path = '/home/jan/projects/irafm/david_mojzisek/Mallet/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=24, id2word=id2word)

In [71]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(16,
  [('distribution', 0.04633277649951146),
   ('inference', 0.044315567182525925),
   ('estimate', 0.031771046742522145),
   ('process', 0.0269486557191036),
   ('probability', 0.025940051060610834),
   ('bayesian', 0.025940051060610834),
   ('probabilistic', 0.025057521984429666),
   ('parameter', 0.024490181864027485),
   ('sample', 0.02345005830995682),
   ('variable', 0.022441453651464053)]),
 (11,
  [('word', 0.1375968992248062),
   ('semantic', 0.07783650458069062),
   ('context', 0.043798449612403104),
   ('lexical', 0.02730796335447498),
   ('corpus', 0.019908386187455954),
   ('sense', 0.01842847075405215),
   ('embedding', 0.01624383368569415),
   ('representation', 0.015539112050739958),
   ('concept', 0.014164904862579281),
   ('verb', 0.014129668780831571)]),
 (5,
  [('image', 0.0424005906549942),
   ('object', 0.03466582287381781),
   ('human', 0.03396266216643814),
   ('recognition', 0.018774390887037234),
   ('visual', 0.014028056112224449),
   ('robot', 0.01311394

In [84]:
from gensim.models.ldamodel import LdaModel
import numpy

def ldaMalletConvertToldaGen(mallet_model):
    model_gensim = LdaModel(
    id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
    alpha=mallet_model.alpha, eta=0, iterations=1000,
    gamma_threshold=0.001,
    dtype=numpy.float32
    )
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [85]:
# transform the mallet Lda model to gensim Lda model
gensim_model = ldaMalletConvertToldaGen(ldamallet)

In [88]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(gensim_model, corpus, id2word)
vis

/home/jan/.conda/envs/mloncode/lib/python3.7/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/jan/.conda/envs/mloncode/lib/python3.7/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/jan/.conda/envs/mloncode/lib/python3.7/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/home/jan/.conda/envs/mloncode/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16     0.142194 -0.061974       1        1  4.736187
3      0.218197  0.001922       2        1  4.485368
6      0.154886 -0.098379       3        1  4.482847
20    -0.154234 -0.159265       4        1  4.449012
9     -0.088788 -0.141584       5        1  4.415973
0     -0.007793  0.090513       6        1  4.358908
1      0.191975 -0.020602       7        1  4.314174
12     0.192083 -0.026155       8        1  4.257223
17     0.129455 -0.139525       9        1  4.214841
18     0.022894 -0.067762      10        1  4.168143
8     -0.152289 -0.138424      11        1  4.149983
22    -0.220714  0.050766      12        1  4.107266
23     0.055700  0.105982      13        1  4.074153
13    -0.025317  0.228882      14        1  4.064610
14    -0.127751 -0.117416      15        1  4.045012
19     0.129942 -0.084282      16        1  4.036441
11    -0.183883 -0.042053      17        1  4.025109
15    -0.030565  0.259383      18        1  4.012151
2     -0.162855  0.011050      19        1  3.985692
10     0.006223  0.160378      20        1  3.965135
21     0.123865  0.108048      21        1  3.951994
5     -0.057253 -0.124338      22        1  3.927354
7     -0.167957  0.112256      23        1  3.904511
4      0.011986  0.092579      24        1  3.867912, topic_info=     Category         Freq             Term        Total  loglift  logprob
120   Default  7577.000000            learn  7577.000000  30.0000  30.0000
383   Default  4887.000000          feature  4887.000000  29.0000  29.0000
165   Default  6132.000000             task  6132.000000  28.0000  28.0000
449   Default  4694.000000             word  4694.000000  27.0000  27.0000
302   Default  6018.000000           system  6018.000000  26.0000  26.0000
156   Default  4352.000000         language  4352.000000  25.0000  25.0000
304   Default  2876.000000      translation  2876.000000  24.0000  24.0000
21    Default  2904.000000           domain  2904.000000  23.0000  23.0000
431   Default  2929.000000         learning  2929.000000  22.0000  22.0000
276   Default  2894.000000        structure  2894.000000  21.0000  21.0000
162   Default  2720.000000         semantic  2720.000000  20.0000  20.0000
384   Default  2509.000000         function  2509.000000  19.0000  19.0000
44    Default  3638.000000      information  3638.000000  18.0000  18.0000
435   Default  2085.000000          network  2085.000000  17.0000  17.0000
559   Default  2097.000000           search  2097.000000  16.0000  16.0000
274   Default  4371.000000              set  4371.000000  15.0000  15.0000
732   Default  2526.000000   representation  2526.000000  14.0000  14.0000
430   Default  1959.000000            label  1959.000000  13.0000  13.0000
360   Default  2163.000000         training  2163.000000  12.0000  12.0000
127   Default  3372.000000      performance  3372.000000  11.0000  11.0000
370   Default  1887.000000   classification  1887.000000  10.0000  10.0000
353   Default  3210.000000            state  3210.000000   9.0000   9.0000
205   Default  2656.000000             time  2656.000000   8.0000   8.0000
182   Default  2337.000000            large  2337.000000   7.0000   7.0000
77    Default  2216.000000         sentence  2216.000000   6.0000   6.0000
85    Default  2699.000000             text  2699.000000   5.0000   5.0000
251   Default  1722.000000       evaluation  1722.000000   4.0000   4.0000
138   Default  2190.000000        technique  2190.000000   3.0000   3.0000
70    Default  1679.000000         relation  1679.000000   2.0000   2.0000
293   Default  2332.000000          machine  2332.000000   1.0000   1.0000
...       ...          ...              ...          ...      ...      ...
3253  Topic24    40.337311     feed_forward    40.337311   3.2525  -6.5711
1559  Topic24   319.427887           smooth   371.317078   3.1019  -4.5019
2330  Topic24    53.419682   